In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import datetime
import pickle
import warnings
import os
import gc
import joblib
import dask
from dask.distributed import Client, LocalCluster
import dask.bag as db
from tqdm import tqdm  # 引入进度条库

warnings.filterwarnings("ignore")

# ==========================================
# 核心配置
# ==========================================
print("正在应用 Dask 通信配置...")
dask.config.set({
    "distributed.comm.timeouts.connect": "180s",
    "distributed.comm.timeouts.tcp": "180s",
    "distributed.comm.retry.count": 5,
    "distributed.worker.memory.target": 0.80,
    "distributed.worker.memory.spill": 0.90,
    "distributed.worker.memory.pause": 0.95
})

# --- 文件路径配置 ---
DATA_DIR = '.' 
CENTER_ROADNET_PATH = os.path.join(DATA_DIR, 'center_roadnet')
CENTER_POS_PATH = os.path.join(DATA_DIR, 'center_pos.npy')
CENTER_REALPOS_PATH = os.path.join(DATA_DIR, 'center_realpos.npy')

COMPLETE_PATH_COORDS_PATH = os.path.join(DATA_DIR, 'Complete_path_coords.pkl')
COMPLETE_PATH_TIME_PATH = os.path.join(DATA_DIR, 'Complete_path_time.pkl')
COMPLETE_PATH_DISTANCE_PATH = os.path.join(DATA_DIR, 'Complete_path_distance.pkl')
COMPLETE_PATH_BAYONET_PATH = os.path.join(DATA_DIR, 'Complete_path_bayonet.pkl')

TRAVEL_INFO_CSV_PATH = os.path.join(DATA_DIR, 'travel_info.csv')
NODES_INFO_PKL_PATH = os.path.join(DATA_DIR, 'nodes_info.pkl')
PATH_RESULT_CSV_PATH = os.path.join(DATA_DIR, 'path_result.csv')
TIME_INTERVAL_DIST_XLSX_PATH = os.path.join(DATA_DIR, '相邻路网节点的时间间隔分布.xlsx')
VEHICLE_INFO_CSV_PATH = os.path.join(DATA_DIR, '中心城区研究车辆信息表.csv')

FILE1_DIR = DATA_DIR 
FILE2_DIR = DATA_DIR 

COMPLETE_PATH_TF_PKL_PATH = os.path.join(FILE1_DIR, 'Complete_path_tf.pkl')
COMPLETE_PATH_POS_PKL_PATH = os.path.join(FILE1_DIR, 'Complete_path_pos.pkl')
PATH_PRED_TRAVEL_PKL_PATH = os.path.join(FILE1_DIR, 'path_pred_travel.pkl')
PATH_POS_NUMBER_PKL_PATH = os.path.join(FILE1_DIR, 'path_pos_number.pkl')

PATH_BAYONET_PKL_PATH = os.path.join(FILE2_DIR, 'final_path_bayonet.pkl')
PATH_TIME_PKL_PATH = os.path.join(FILE2_DIR, 'final_path_time.pkl')
PATH_START_TIME_PKL_PATH = os.path.join(FILE2_DIR, 'final_path_start_time.pkl')
PATH_STOP_TIME_PKL_PATH = os.path.join(FILE2_DIR, 'final_path_stop_time.pkl')
PATH_TRAVEL_PKL_PATH = os.path.join(FILE2_DIR, 'final_path_travel.pkl')
PATH_CLTMBH_PKL_PATH = os.path.join(FILE2_DIR, 'final_path_cltmbh.pkl')
PATH_BAYONET_NUMBER_NPY_PATH = os.path.join(FILE2_DIR, 'path_bayonet_number.npy')

def load_raw_data():
    print("正在加载基础数据...")
    global G, pos_coord, pos_index_map, bayonet_name, realpos_index_map
    G = nx.read_gml(CENTER_ROADNET_PATH)
    pos = np.load(CENTER_POS_PATH, allow_pickle=True).item()
    pos_name = list(pos.keys())
    pos_coord = list(pos.values())
    realpos = np.load(CENTER_REALPOS_PATH, allow_pickle=True)
    realpos_name = list(realpos[0])
    bayonet_name = list(realpos[1])
    pos_index_map = {name: idx for idx, name in enumerate(pos_name)}
    realpos_index_map = {name: idx for idx, name in enumerate(realpos_name)}

    with open(COMPLETE_PATH_TF_PKL_PATH, 'rb') as f: Complete_path_tf = pickle.load(f)
    with open(COMPLETE_PATH_POS_PKL_PATH, 'rb') as f: Complete_path_pos = pickle.load(f)
    with open(PATH_PRED_TRAVEL_PKL_PATH, 'rb') as f: path_pred_travel = pickle.load(f)
    with open(PATH_POS_NUMBER_PKL_PATH, 'rb') as f: path_pos_number = pickle.load(f)

    path_bayonet = np.load(PATH_BAYONET_PKL_PATH, allow_pickle=True)
    path_time = np.load(PATH_TIME_PKL_PATH, allow_pickle=True)
    path_start_time = np.load(PATH_START_TIME_PKL_PATH, allow_pickle=True)
    path_stop_time = np.load(PATH_STOP_TIME_PKL_PATH, allow_pickle=True)
    path_travel = np.load(PATH_TRAVEL_PKL_PATH, allow_pickle=True)
    path_cltmbh = np.load(PATH_CLTMBH_PKL_PATH, allow_pickle=True)
    path_bayonet_number = np.load(PATH_BAYONET_NUMBER_NPY_PATH, allow_pickle=True)
    return (Complete_path_tf, Complete_path_pos, path_travel, path_pred_travel, 
            path_pos_number, path_bayonet_number, path_cltmbh, 
            path_start_time, path_stop_time, path_time, path_bayonet)

def calculate_single_trip_v2(args):
    idx, status, path, travel_seconds, raw_start_time, raw_end_time = args
    res = {'idx': idx, 'status': status, 'valid': False, 'nodes': None}
    if status != "get" or not isinstance(path, list) or len(path) == 0: return res
    try:
        total_dist = 0
        step_dists = [0]
        if len(path) > 1:
            for u, v in zip(path[:-1], path[1:]):
                d = G.edges[u, v]['length']
                total_dist += d
                step_dists.append(round(d, 2))
        total_dist = round(total_dist, 2)
        velocity = round((total_dist / travel_seconds) * 3.6, 2) if travel_seconds > 0 else np.nan
        coords = [pos_coord[pos_index_map[p]] for p in path]
        bayonets = [bayonet_name[realpos_index_map[p]] if p in realpos_index_map else np.nan for p in path]
        times = [raw_start_time]
        current_time = raw_start_time
        if total_dist > 0:
            for d in step_dists[1:]:
                rate = d / total_dist
                add_sec = max(0, round(travel_seconds * rate))
                current_time += datetime.timedelta(seconds=add_sec)
                times.append(current_time)
        else:
            times = [raw_start_time] * len(path)
        times[-1] = raw_end_time
        res.update({'total_distance': total_dist, 'velocity': velocity, 'step_distances': step_dists,
                    'step_coords': coords, 'step_bayonets': bayonets, 'step_times': times,
                    'nodes': path, 'valid': True})
        return res
    except Exception: return res

def flatten_trip_to_rows(trip_data, cltmbh):
    if not trip_data['valid']: return []
    rows = []
    c_distance = 0
    travel_id = trip_data['idx']
    dists, times, bays, coords, nodes = trip_data['step_distances'], trip_data['step_times'], trip_data['step_bayonets'], trip_data['step_coords'], trip_data['nodes']
    for i in range(len(nodes)):
        dist = dists[i]
        c_distance += dist
        rows.append({'device_id': cltmbh, 'travel_id': travel_id, 'datetime': times[i],
                     'bayonetname': bays[i], 'posname': nodes[i], 'longitude': coords[i][0],
                     'latitude': coords[i][1], 'c_distance': c_distance, 'distance': dist})
    return rows

def main():
    # 为了保证进度条显示正确，不显示 Dask 内部的进度条
    # from dask.diagnostics import ProgressBar  <-- 移除这个，用 tqdm 代替

    print(f"正在启动 Dask Client (Core: 7 Worker, 2 System)...")
    cluster = LocalCluster(n_workers=7, threads_per_worker=1, memory_limit='24GB')
    client = Client(cluster)
    print(f"Dask Dashboard: {client.dashboard_link}")

    (Complete_path_tf, Complete_path_pos, path_travel, path_pred_travel, path_pos_number, 
     path_bayonet_number, path_cltmbh, path_start_time, path_stop_time, path_time, path_bayonet) = load_raw_data()

    print("正在构建任务队列...")
    tasks = []
    total = len(Complete_path_tf)
    for i in range(total):
        if isinstance(path_time[i], list) and len(path_time[i]) > 0: s_time, e_time = path_time[i][0], path_time[i][-1]
        else: s_time, e_time = path_start_time[i], path_stop_time[i]
        tasks.append((i, Complete_path_tf[i], Complete_path_pos[i], path_travel[i], s_time, e_time))

    print(f"开始并行处理 {total} 条路径 (Batch模式)...")
    
    # 1. 任务入袋 & 挂载函数
    bag = db.from_sequence(tasks, npartitions=100).map(calculate_single_trip_v2)
    
    # 2. 转换为 Delayed 列表
    delayed_list = bag.to_delayed()
    
    results = []
    batch_size = 10 
    
    # ==========================================
    # 【修改点】引入 tqdm 全局进度条
    # ==========================================
    # tqdm 会自动计算总批次数，并显示剩余时间
    # range(0, len(delayed_list), batch_size) 决定了循环次数
    
    for i in tqdm(range(0, len(delayed_list), batch_size), desc="全局处理进度", unit="batch"):
        batch = delayed_list[i : i + batch_size]
        
        # 执行计算
        batch_results = dask.compute(*batch)
        
        for part in batch_results: 
            results.extend(part)
        
        gc.collect()

    print(f"计算完成，共获取结果 {len(results)} 条")
    
    # 结果排序
    results.sort(key=lambda x: x['idx'])
    
    rec_dist, rec_coords, rec_bayonet, rec_time = [], [], [], []
    travel_info_rows, valid_results = [], []
    
    for i, res in enumerate(results):
        if res['valid']:
            rec_dist.append(res['step_distances'])
            rec_coords.append(res['step_coords'])
            rec_bayonet.append(res['step_bayonets'])
            rec_time.append(res['step_times'])
            travel_info_rows.append({'CLTMBH': path_cltmbh[i], 'start_time': path_start_time[i],
                'stop_time': path_stop_time[i], 'travel_time_s': path_travel[i], 'predtravel_time_s': path_pred_travel[i],
                'distance_km': round(res['total_distance'] / 1000, 2), 'speed_kmh': res['velocity'],
                'bayonet_records': path_bayonet_number[i], 'node_records': path_pos_number[i],
                'start_bayonet': res['step_bayonets'][0], 'stop_bayonet': res['step_bayonets'][-1],
                'start_node_lon': res['step_coords'][0][0], 'start_node_lat': res['step_coords'][0][1],
                'stop_node_lon': res['step_coords'][-1][0], 'stop_node_lat': res['step_coords'][-1][1], 'position': i})
            valid_results.append(res)
        else:
            rec_dist.append(np.nan); rec_coords.append(np.nan); rec_bayonet.append(np.nan); rec_time.append(np.nan)

    print("正在保存修复后的中间文件...")
    joblib.dump(rec_dist, COMPLETE_PATH_DISTANCE_PATH)
    joblib.dump(rec_coords, COMPLETE_PATH_COORDS_PATH)
    joblib.dump(rec_bayonet, COMPLETE_PATH_BAYONET_PATH)
    joblib.dump(rec_time, COMPLETE_PATH_TIME_PATH)

    print("正在生成 travel_info.csv ...")
    travel_df = pd.DataFrame(travel_info_rows)
    if os.path.exists(VEHICLE_INFO_CSV_PATH):
        veh_info = pd.read_csv(VEHICLE_INFO_CSV_PATH, encoding='gbk')
        travel_df = pd.merge(travel_df, veh_info[['CLTMBH', 'veh_type', 'fuel_type', 'PFBZ']], on='CLTMBH', how='left')
    
    travel_df = travel_df[(travel_df['speed_kmh'] <= 60) & (travel_df['distance_km'] >= 0.5)].copy()
    def check_error(row):
        a, b = row['predtravel_time_s'], row['travel_time_s']
        if pd.isna(a) or pd.isna(b) or b <= 0: return False
        return (b * 0.4) < a < (b * 1.5)
    travel_df = travel_df[travel_df.apply(check_error, axis=1)]
    travel_df.to_csv(TRAVEL_INFO_CSV_PATH, encoding='gbk', index=False)
    
    valid_positions = travel_df['position'].tolist()
    nodes_info = [[Complete_path_pos[i] for i in valid_positions], [rec_coords[i] for i in valid_positions],
                  [rec_time[i] for i in valid_positions], [rec_dist[i] for i in valid_positions], [rec_bayonet[i] for i in valid_positions]]
    with open(NODES_INFO_PKL_PATH, 'wb') as f: pickle.dump(nodes_info, f)

    print("正在生成 path_result.csv ...")
    valid_pos_set = set(valid_positions)
    final_explode_tasks = [(res, path_cltmbh[res['idx']]) for res in valid_results if res['idx'] in valid_pos_set]
    bag_explode = db.from_sequence(final_explode_tasks, npartitions=54)
    ddf = bag_explode.map(lambda args: flatten_trip_to_rows(*args)).flatten().to_dataframe()
    ddf.to_csv(PATH_RESULT_CSV_PATH, index=False, single_file=True, encoding='gbk')

    print("正在生成时间间隔统计...")
    all_intervals = []
    for times in rec_time:
        if isinstance(times, list) and len(times) > 1: all_intervals.extend([(times[j]-times[j-1]).seconds for j in range(1, len(times))])
    if all_intervals:
        interval_bins = [0, 5, 15, 30, 60, 300, 900, 1800, 3600, 28800, 43200, 86400]
        res_cut = pd.cut(all_intervals, bins=interval_bins, labels=interval_bins[:-1], right=False)
        pd.value_counts(res_cut, sort=False).reset_index().to_excel(TIME_INTERVAL_DIST_XLSX_PATH, index=False)

    print("🎉 所有任务全部完成！")
    client.close()

if __name__ == '__main__':
    main()

正在应用 Dask 通信配置...
正在启动 Dask Client (Core: 7 Worker, 2 System)...
Dask Dashboard: http://127.0.0.1:8787/status
正在加载基础数据...
正在构建任务队列...
开始并行处理 3009117 条路径 (Batch模式)...


全局处理进度: 100%|██████████| 11/11 [38:32<00:00, 210.24s/batch]


计算完成，共获取结果 3009117 条
正在保存修复后的中间文件...


2025-12-01 19:30:34,573 - distributed.nanny - WARNING - Restarting worker


In [ ]:
!pip install "bokeh>=3.1.0"

In [ ]:
client.close()